In [3]:
import sklearn_crfsuite
from type import CrfFeatures, Abbreviation
import json

In [4]:
# 前処理した略語データを読み込む
data = list(map(Abbreviation.model_validate, json.load(open("./data/abbreviation.json", "r"))))

In [ ]:
model = sklearn_crfsuite.CRF()